In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import pandas as pd
import numpy as np
import os
import json

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
df_bbidding.columns.to_list()

['timestamp_utc',
 'Mean_SolarRadiation_dwd',
 'SolarDownwardRadiation_RW_dwd_Mean_30min',
 'SolarDownwardRadiation_RW_dwd_Mean_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_30min',
 'SolarDownwardRadiation_dwd_Mean_Lag_1h',
 'SolarDownwardRadiation_dwd_Mean_Lag_24h',
 'Panel_Efficiency_dwd_mean',
 'Panel_Efficiency_dwd_std',
 'Panel_Temperature_dwd_mean',
 'Panel_Temperature_dwd_std',
 'Std_Temperature_dwd',
 'Mean_Temperature_dwd',
 'cos_hour',
 'cos_day',
 'solar_mw_lag_48h',
 'capacity_mwp_lag_48h',
 'Target_Capacity_MWP%_lag_48h',
 'Target_Capacity_MWP%',
 'Solar_MWh_credit',
 'reference_time',
 'valid_time',
 'RelativeHumidity_dwd',
 'Temperature_dwd',
 'WindDirection_dwd',
 'WindDirection:100_dwd',
 'WindSpeed^3_dwd',
 'WindSpeed:100^3_dwd',
 'WindSpeed_dwd',
 'WindSpeed:100_dwd',
 'WindSpeed^3:100_dwd',
 'RelativeHumidity_ncep',
 'Temperature_ncep',
 'WindDirection_ncep',
 'WindDirection:100_ncep',
 'WindSpeed^3_ncep',
 'WindSpeed:100^3_ncep',
 'WindSpeed_ncep',
 'WindSpeed:100_nc

In [3]:
df_bbidding1 = df_bbidding[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_bbidding1.loc[:,"market_price_lag48h"] = df_bbidding1["market_price"].shift(96)
df_bbidding1.loc[:,"imbalance_price_lag48h"] = df_bbidding1["imbalance_price"].shift(96)
df_bbidding1.loc[:,"day_ahead_price_lag1week"] = df_bbidding1["day_ahead_price"].shift(336)
df_bbidding1.loc[:,"volume_lag48h"] = df_bbidding1["volume"].shift(96)
df_bbidding1.dropna(inplace=True)
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h
96,2021-02-22 23:00:00+00:00,39.86,40.74,850.50,47,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088636,...,1.146678e-09,2.732664e-20,40.952962,-0.658593,-4.207329,39.86000,26.46,32.75000,32.38,671.75
97,2021-02-22 23:30:00+00:00,25.88,40.74,755.65,48,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088686,...,1.146678e-09,2.732664e-20,53.022213,14.302050,2.706285,25.88000,29.81,62.50000,32.38,905.45
98,2021-02-23 00:00:00+00:00,31.95,35.25,821.25,1,1.000000,0.598181,-2.668594,1.327215e+00,-7.243929,...,1.336674e+00,-5.844249e+01,7.936466,44.084019,84.017522,31.95000,7.67,11.79015,17.34,781.05
99,2021-02-23 00:30:00+00:00,32.09,35.25,852.25,2,1.000000,0.598181,-83.386905,-4.611063e+01,-92.411183,...,1.028806e+00,-6.628035e+01,50.825184,23.217360,89.462875,32.18992,9.82,12.29000,17.34,694.45
100,2021-02-23 01:00:00+00:00,30.75,32.74,789.00,3,0.965926,0.598181,-92.621997,-3.236547e+01,-51.507430,...,5.531250e+01,4.985395e+01,94.284233,109.047937,123.758802,30.55000,10.40,16.88000,14.25,847.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,8.558464e+01,147.450968,...,2.575452e+02,3.120008e+02,389.209655,461.471346,565.748439,66.01000,0.00,70.00000,87.62,0.00
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,4.406274e+01,80.500728,...,1.529461e+02,2.133404e+02,285.856261,352.111989,494.824570,66.01000,0.00,72.00000,85.84,0.00
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-3.103038e+01,1.674412,...,8.055552e+01,1.386240e+02,177.132223,250.514362,333.223882,66.01000,0.00,71.00000,85.84,0.00
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-3.898848e+01,19.495393,...,7.219972e+01,1.286610e+02,157.102188,214.652927,345.416021,66.01000,83.73,98.00000,82.02,194.45


In [4]:
# Original dataset
X = df_bbidding1[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
y = df_bbidding1["day_ahead_price"].values  # Convert to numpy

# Step 1: Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 2: Further split the training set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# Step 3: Standardize the data (use only training data to fit the scaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [6]:
# y_train = y_train.values
# y_test = y_test.values

In [7]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Step 4: Create TensorDataset and DataLoader for training, validation, and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1                  # Dropout rate
learning_rate = 0.0001         # Learning rate for optimizer
batch_size = 32                # Batch size
num_epochs = 500               # Maximum number of epochs
patience = 15                  # Patience for early stopping
rel_improvement_threshold = 0.0001  # Relative improvement threshold

In [8]:
input_size

15

In [9]:
hidden_size

64

In [10]:
model = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

best_loss = float('inf')
best_model = None
patience_counter = 0
previous_loss = float('inf')

In [11]:
import torch.nn.functional as F  # For MAE loss function

best_val_loss = float('inf')  # Initialize the best validation loss
best_model = None
patience_counter = 0  # For early stopping

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension (for some models)
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the loss using MAE
        loss = F.l1_loss(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

    epoch_train_loss = running_train_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}')

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Reshape input to add sequence length dimension
            X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

            # Forward pass
            y_pred = model(X_batch)  # (batch_size, output_size)

            # Compute the validation loss using MAE
            val_loss = F.l1_loss(y_pred, y_batch)
            running_val_loss += val_loss.item()

    epoch_val_loss = running_val_loss / len(val_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}')

    # Early stopping based on validation loss
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_model = model.state_dict()  # Save the best model's weights
        patience_counter = 0  # Reset patience counter
        print(f"Best model saved with validation loss: {best_val_loss:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

# Load the best model
if best_model is not None:
    model.load_state_dict(best_model)
else:
    print("No improvement was observed during training.")

torch.save(model.state_dict(), 'LSTM_day_ahead_price.pth')

# Test the model using the best model
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Reshape input to add sequence length dimension
        X_batch = X_batch.unsqueeze(1)  # (batch_size, 1, input_size)

        # Forward pass
        y_pred = model(X_batch)  # (batch_size, output_size)

        # Compute the test loss using MAE
        loss = F.l1_loss(y_pred, y_batch)
        test_loss += loss.item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}')


Epoch [1/500], Training Loss: 166.3687
Epoch [1/500], Validation Loss: 143.0704
Best model saved with validation loss: 143.0704
Epoch [2/500], Training Loss: 157.3330
Epoch [2/500], Validation Loss: 137.2172
Best model saved with validation loss: 137.2172
Epoch [3/500], Training Loss: 152.3789
Epoch [3/500], Validation Loss: 132.8330
Best model saved with validation loss: 132.8330
Epoch [4/500], Training Loss: 148.0261
Epoch [4/500], Validation Loss: 128.7217
Best model saved with validation loss: 128.7217
Epoch [5/500], Training Loss: 143.8372
Epoch [5/500], Validation Loss: 124.7328
Best model saved with validation loss: 124.7328
Epoch [6/500], Training Loss: 139.7191
Epoch [6/500], Validation Loss: 120.8081
Best model saved with validation loss: 120.8081
Epoch [7/500], Training Loss: 135.6472
Epoch [7/500], Validation Loss: 116.9451
Best model saved with validation loss: 116.9451
Epoch [8/500], Training Loss: 131.6183
Epoch [8/500], Validation Loss: 113.1323
Best model saved with va

In [12]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [13]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)
df_api_new

,prediction_date,timestamp,1,2,3,4,5,6,7,8,9
0,2024-10-04,2024-10-03T22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,1357
1,2024-10-04,2024-10-03T22:30:00+00:00,788,584,1148,1320,1007,565,923,797,487
2,2024-10-04,2024-10-03T23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,991
3,2024-10-04,2024-10-03T23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,463
4,2024-10-04,2024-10-04T00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,1340
...,...,...,...,...,...,...,...,...,...,...,...
811,2024-10-19,2024-10-19T19:30:00+00:00,3,7,23,32,45,57,77,93,125
812,2024-10-19,2024-10-19T20:00:00+00:00,0,11,25,36,42,60,82,103,132
813,2024-10-19,2024-10-19T20:30:00+00:00,0,16,27,39,44,64,89,106,140
814,2024-10-19,2024-10-19T21:00:00+00:00,0,11,29,42,51,75,97,113,151


In [14]:
df_api_new["timestamp"] = pd.to_datetime(df_api_new["timestamp"])
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)
df_api_new_merged

,prediction_date,timestamp,1,2,3,4,5,6,7,8,...,price_y,volume,timestamp_utc,settlement_date,settlement_period,imbalance_price,day_ahead_price,market_price,cos_hour,cos_day
0,2024-10-04,2024-10-03 22:00:00+00:00,1355,487,963,1544,1330,867,326,1402,...,71.32,1322.45,2024-10-03 22:00:00+00:00,2024-10-03,47.0,61.0,78.47,71.32,0.866025,-0.900969
1,2024-10-04,2024-10-03 22:30:00+00:00,788,584,1148,1320,1007,565,923,797,...,71.42,1035.65,2024-10-03 22:30:00+00:00,2024-10-03,48.0,50.0,78.47,71.42,0.866025,-0.900969
2,2024-10-04,2024-10-03 23:00:00+00:00,571,748,1001,1460,740,1017,1533,597,...,66.94,1378.55,2024-10-03 23:00:00+00:00,2024-10-04,1.0,93.5,71.36,66.94,0.965926,-0.900969
3,2024-10-04,2024-10-03 23:30:00+00:00,1476,1194,1512,334,1054,1572,744,1497,...,60.03,1539.90,2024-10-03 23:30:00+00:00,2024-10-04,2.0,93.5,71.36,60.03,0.965926,-0.900969
4,2024-10-04,2024-10-04 00:00:00+00:00,1352,1390,912,662,1505,1093,814,487,...,65.21,1818.40,2024-10-04 00:00:00+00:00,2024-10-04,3.0,93.0,72.80,65.21,1.000000,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,2024-10-19,2024-10-19 19:30:00+00:00,3,7,23,32,45,57,77,93,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.258819,-0.222521
819,2024-10-19,2024-10-19 20:00:00+00:00,0,11,25,36,42,60,82,103,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.222521
820,2024-10-19,2024-10-19 20:30:00+00:00,0,16,27,39,44,64,89,106,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.500000,-0.222521
821,2024-10-19,2024-10-19 21:00:00+00:00,0,11,29,42,51,75,97,113,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0.707107,-0.222521


In [15]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag48h"] = df_api_new_merged1["market_price"].shift(96)
df_api_new_merged1.loc[:,"imbalance_price_lag48h"] = df_api_new_merged1["imbalance_price"].shift(96)
df_api_new_merged1.loc[:,"day_ahead_price_lag48h"] = df_api_new_merged1["day_ahead_price"].shift(96)
df_api_new_merged1.loc[:,"volume_lag48h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,5,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h
96,2024-10-08 21:30:00+00:00,55.09,70.40,2360.40,46.0,0.707107,0.62349,18,17,16,...,10,8,4,3,1,55.09,71.32,61.00,78.47,1322.45
97,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.00000,19,17,15,...,10,8,6,3,1,100.00,71.42,50.00,78.47,1035.65
98,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.00000,19,17,15,...,10,8,6,3,1,66.00,66.94,93.50,71.36,1378.55
99,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.00000,19,17,15,...,10,8,5,2,1,66.00,60.03,93.50,71.36,1539.90
100,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.00000,19,17,15,...,10,8,5,2,1,66.00,65.21,93.00,72.80,1818.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.62349,850,921,980,...,1060,1081,1104,1123,1142,105.00,203.10,124.71,91.30,318.15
531,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.62349,897,964,1010,...,1074,1097,1111,1129,1151,104.90,152.35,74.72,85.25,289.25
532,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.62349,910,984,1040,...,1101,1120,1122,1139,1154,64.00,118.88,74.72,85.25,541.20
533,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.62349,938,1034,1069,...,1124,1135,1139,1152,1164,33.77,103.72,102.00,91.58,1282.20


In [16]:
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()

In [17]:
X_test = df_api_new_merged1[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X_test = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
X_test_tensor.shape

torch.Size([334, 15])

In [18]:
X_test_tensor = X_test_tensor.unsqueeze(1)  # Adds a sequence length dimension


In [19]:


model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_outputs = final_test_outputs.numpy().flatten()

df_api_new_merged1["day_ahead_price_predictions"] = final_test_outputs
df_api_new_merged1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h,day_ahead_price_predictions
0,2024-10-07 22:00:00+00:00,82.03,89.55,964.20,47.0,0.866025,1.00000,19,17,15,...,8,6,3,1,100.00,71.42,50.00,78.47,1035.65,88.811287
1,2024-10-07 22:30:00+00:00,84.67,89.55,1265.00,48.0,0.866025,1.00000,19,17,15,...,8,6,3,1,66.00,66.94,93.50,71.36,1378.55,74.437347
2,2024-10-07 23:00:00+00:00,79.56,87.15,1112.60,1.0,0.965926,1.00000,19,17,15,...,8,5,2,1,66.00,60.03,93.50,71.36,1539.90,70.164833
3,2024-10-07 23:30:00+00:00,75.63,87.15,1240.55,2.0,0.965926,1.00000,19,17,15,...,8,5,2,1,66.00,65.21,93.00,72.80,1818.40,71.996056
4,2024-10-08 00:00:00+00:00,74.84,81.06,1297.45,3.0,1.000000,0.62349,18,16,14,...,7,5,3,1,65.00,64.79,93.00,72.80,1784.30,64.849022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,2024-10-15 19:30:00+00:00,88.67,80.72,2071.15,42.0,0.258819,0.62349,850,921,980,...,1081,1104,1123,1142,105.00,203.10,124.71,91.30,318.15,129.504013
330,2024-10-15 20:00:00+00:00,83.31,64.10,1709.85,43.0,0.500000,0.62349,897,964,1010,...,1097,1111,1129,1151,104.90,152.35,74.72,85.25,289.25,112.671600
331,2024-10-15 20:30:00+00:00,69.98,64.10,1551.30,44.0,0.500000,0.62349,910,984,1040,...,1120,1122,1139,1154,64.00,118.88,74.72,85.25,541.20,103.992027
332,2024-10-15 21:00:00+00:00,69.82,28.87,1055.25,45.0,0.707107,0.62349,938,1034,1069,...,1135,1139,1152,1164,33.77,103.72,102.00,91.58,1282.20,104.618744


In [20]:
#plot imbalance price and predictions
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_api_new_merged1["timestamp_utc"], y=df_api_new_merged1["day_ahead_price"], mode='lines', name='Imbalance Price'))
fig.add_trace(go.Scatter(x=df_api_new_merged1["timestamp_utc"], y=df_api_new_merged1["day_ahead_price_predictions"], mode='lines', name='Imbalance Price Predictions'))
fig.update_layout(title='Imbalance Price and Predictions', xaxis_title='Timestamp', yaxis_title='Imbalance Price')
fig.show()

In [21]:
#MAE calculation for imbalance price predictions
mae = np.mean(np.abs(df_api_new_merged1["day_ahead_price"] - df_api_new_merged1["day_ahead_price_predictions"]))
print(f"MAE for imbalance price predictions: {mae:.4f}")

MAE for imbalance price predictions: 29.5838


In [22]:
#prediction for the whole dataset
X = df_bbidding1[["market_price_lag48h","imbalance_price_lag48h","day_ahead_price_lag48h","volume_lag48h",
                  "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].values
X = scaler.transform(X)
X_tensor = torch.tensor(X, dtype=torch.float32)
X_tensor = X_tensor.unsqueeze(1)  # Adds a sequence length

model.eval()
with torch.no_grad():
    final_outputs = model(X_tensor)
    final_outputs = final_outputs.numpy().flatten()

df_bbidding1["day_ahead_price_predictions"] = final_outputs
df_bbidding1

,timestamp_utc,market_price,day_ahead_price,volume,settlement_period,cos_hour,cos_day,1,2,3,...,6,7,8,9,imbalance_price,market_price_lag48h,imbalance_price_lag48h,day_ahead_price_lag48h,volume_lag48h,day_ahead_price_predictions
96,2021-02-22 23:00:00+00:00,39.86,40.74,850.50,47,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088636,...,2.732664e-20,40.952962,-0.658593,-4.207329,39.86000,26.46,32.75000,32.38,671.75,43.437901
97,2021-02-22 23:30:00+00:00,25.88,40.74,755.65,48,0.965926,0.611886,-0.000088,-9.562141e-08,-0.088686,...,2.732664e-20,53.022213,14.302050,2.706285,25.88000,29.81,62.50000,32.38,905.45,43.062424
98,2021-02-23 00:00:00+00:00,31.95,35.25,821.25,1,1.000000,0.598181,-2.668594,1.327215e+00,-7.243929,...,-5.844249e+01,7.936466,44.084019,84.017522,31.95000,7.67,11.79015,17.34,781.05,43.795162
99,2021-02-23 00:30:00+00:00,32.09,35.25,852.25,2,1.000000,0.598181,-83.386905,-4.611063e+01,-92.411183,...,-6.628035e+01,50.825184,23.217360,89.462875,32.18992,9.82,12.29000,17.34,694.45,43.883202
100,2021-02-23 01:00:00+00:00,30.75,32.74,789.00,3,0.965926,0.598181,-92.621997,-3.236547e+01,-51.507430,...,4.985395e+01,94.284233,109.047937,123.758802,30.55000,10.40,16.88000,14.25,847.35,44.415764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42280,2023-08-25 23:30:00+00:00,83.91,84.41,1201.85,2,0.965926,-0.591261,28.044030,8.558464e+01,147.450968,...,3.120008e+02,389.209655,461.471346,565.748439,66.01000,0.00,70.00000,87.62,0.00,97.826805
42281,2023-08-26 00:00:00+00:00,82.13,79.96,1066.95,3,1.000000,-0.577292,-2.130944,4.406274e+01,80.500728,...,2.133404e+02,285.856261,352.111989,494.824570,66.01000,0.00,72.00000,85.84,0.00,96.985741
42282,2023-08-26 00:30:00+00:00,78.76,79.96,997.60,4,1.000000,-0.577292,-53.850807,-3.103038e+01,1.674412,...,1.386240e+02,177.132223,250.514362,333.223882,66.01000,0.00,71.00000,85.84,0.00,96.886719
42283,2023-08-26 01:00:00+00:00,77.47,73.84,913.25,5,0.965926,-0.577292,-42.706652,-3.898848e+01,19.495393,...,1.286610e+02,157.102188,214.652927,345.416021,66.01000,83.73,98.00000,82.02,194.45,101.025780


In [23]:
df_bbidding1.to_csv('bidding_training_predictions_day_ahead_price.csv', index=False)